# SEO Crawl Analysis Template

After running the [`crawl`](https://advertools.readthedocs.io/en/master/advertools.spider.html#advertools.spider.crawl) function and saving your output file, you need to take a look and start your analysis. This is meant to be template for an initial exploration of the data that is typically available in the output.  
We will go through the default columns, conventions in storing and naming data, and exploratory techniques that you will probably also want to run during your first exploration of the dataset.

In [1]:
import advertools as adv
import pandas as pd
pd.options.display.max_columns = None

The site chosen for this template was because it had a few thousand pages (not too few, and not too many), and it has a simple HTML structure, so it was striaghtforward to extract the required elements. Many times you will face different kinds of issues when crawling and scraping pages. I'm sure you've heard of [The White House](https://www.whitehouse.gov/).

### Robots.txt

We are pretending to be Google, right?  
So we start by getting the robotstxt file. 

In [2]:
# robots_df = adv.robotstxt_to_df('https://www.whitehouse.gov/robots.txt')

robots_df = pd.read_csv('robotstxt_df.csv')
robots_df

,directive,content,robotstxt_url,file_downloaded
0,User-agent,*,https://www.whitehouse.gov/robots.txt,2020-05-28 14:32:09.827296+00:00
1,Disallow,/wp-admin/,https://www.whitehouse.gov/robots.txt,2020-05-28 14:32:09.827296+00:00
2,Allow,/wp-admin/admin-ajax.php,https://www.whitehouse.gov/robots.txt,2020-05-28 14:32:09.827296+00:00


Not much information here. But we will save it for later crawls where things might change.

### Sitemaps

Now we get the sitemaps using the `sitemap_to_df` function:

In [3]:
# sitemap_df = adv.sitemap_to_df('https://www.whitehouse.gov/sitemap_index.xml')

sitemap_df = pd.read_csv('sitemap_df.csv', parse_dates=['lastmod'])
sitemap_df.sample(5)

,loc,lastmod,image,image_loc,image_caption,image_title,sitemap,sitemap_downloaded
6959,https://www.whitehouse.gov/presidential-action...,2018-01-09 20:07:07+00:00,NaN,NaN,NaN,NaN,https://www.whitehouse.gov/presidential-action...,2020-05-26 23:31:56.770463+00:00
3077,https://www.whitehouse.gov/briefings-statement...,2018-04-09 16:30:57+00:00,NaN,NaN,NaN,NaN,https://www.whitehouse.gov/briefing-statement-...,2020-05-26 23:31:56.770463+00:00
6061,https://www.whitehouse.gov/briefings-statement...,2020-01-30 22:26:54+00:00,NaN,NaN,NaN,NaN,https://www.whitehouse.gov/briefing-statement-...,2020-05-26 23:31:56.770463+00:00
2071,https://www.whitehouse.gov/briefings-statement...,2017-12-07 21:29:41+00:00,NaN,NaN,NaN,NaN,https://www.whitehouse.gov/briefing-statement-...,2020-05-26 23:31:56.770463+00:00
4648,https://www.whitehouse.gov/briefings-statement...,2019-03-01 02:28:27+00:00,NaN,NaN,NaN,NaN,https://www.whitehouse.gov/briefing-statement-...,2020-05-26 23:31:56.770463+00:00


A quick look at the frequency of content publishing shows that they only started in 2017 with the following annual numer of pages published:

In [4]:
sitemap_df.set_index('lastmod').resample('A')['loc'].count()

lastmod
2017-12-31 00:00:00+00:00    2446
2018-12-31 00:00:00+00:00    2702
2019-12-31 00:00:00+00:00    2111
2020-12-31 00:00:00+00:00     929
Name: loc, dtype: int64

We might also get a clue regarding how the content is distributed across the sitemaps:

In [5]:
sitemap_df['sitemap'].value_counts()

https://www.whitehouse.gov/briefing-statement-sitemap1.xml     1001
https://www.whitehouse.gov/presidential-action-sitemap1.xml    1001
https://www.whitehouse.gov/briefing-statement-sitemap5.xml     1000
https://www.whitehouse.gov/briefing-statement-sitemap2.xml     1000
https://www.whitehouse.gov/briefing-statement-sitemap4.xml     1000
https://www.whitehouse.gov/briefing-statement-sitemap3.xml     1000
https://www.whitehouse.gov/briefing-statement-sitemap6.xml      838
https://www.whitehouse.gov/post-sitemap.xml                     567
https://www.whitehouse.gov/presidential-action-sitemap2.xml     476
https://www.whitehouse.gov/page-sitemap.xml                     154
https://www.whitehouse.gov/eop-component-sitemap.xml             88
https://www.whitehouse.gov/authors-sitemap.xml                   27
https://www.whitehouse.gov/microsite-sitemap.xml                 24
https://www.whitehouse.gov/issue-sitemap.xml                     13
Name: sitemap, dtype: int64

It seems the majority of pages are for briefings and statements, and the second biggest topic is presidential actions. We also have a few generic pages.  
If you check any of those pages, you'll see how simple they are, and how straightforward it is to extract the required data from them. The `crawl` function returns a standard set of page elements, but it also allows you to add any columns you like, name them whatever you want and extract any data by using CSS and/or XPath selectors.  You can do this by passing dictionary to either (or both) of those parameters (`css_selectors` and `xpath_selectors`).  
Looking at both types of pages (briefings and actions) I realized that they each have four main elements in their pages that might be interesting; The title, the date, the category, and the body text of the content. So I created this dictionary to extract them.  
The keys of this dictionary will become column names, and their values will be the selectors used to extrat those data from the pages (if they are available and `NaN` otherwise). I named the keys `briefing_*` but they will be valid for both briefings and presidential actions.

In [6]:
selectors = {'briefing_title': '.page-header__title::text',
             'briefing_date': 'time::text',
             'briefing_body_text': '.editor p::text',
             'briefing_category': '.issue-flag a::text'}

`body_text` is already a standard element that is provided by the `crawl` function. The problem is that by default it cannot know what exactly constitutes the "body text" of a page. So it extracts all the \<p>, \<span>, and \<li> elements within \<body> tags from pages, which might contain some additional noise. So in this case I selected a specific CSS selector for that. 

## Crawling

Now that we have the full list of the sitemap URLs we have two options: 
1. **Crawl the sitemap URLs:** This is a straightforward approach, and goes through the given URLs only, which I have done in this example. This approach is good if you know exactly what you are looking for (and not checking if there are hidden pages, or issues with the sitemaps). You might use this for monitoring purposes as well.
2. **Crawl the website starting from the home page and following links (`follow_links=True`):** This might be a better approach for an SEO audit because you crawl independently from the sitemap and can then compare the URLs you were able to discover through links with what is available in the sitemap. You might discover pages that are not in the sitemaps and vice versa, and this can really help in understanding some issues you might come across. I recommend the second approach for this situation, but I did the first one as this is just to demonstrate a way for exploring the output file.  

Now, we'll take the unique sitemaps, crawl each set of URLs separately, and save each output to a different file. 

In [7]:
sitemaps_unique = sitemap_df['sitemap'].drop_duplicates().tolist()
sitemaps_unique

['https://www.whitehouse.gov/post-sitemap.xml',
 'https://www.whitehouse.gov/page-sitemap.xml',
 'https://www.whitehouse.gov/briefing-statement-sitemap1.xml',
 'https://www.whitehouse.gov/briefing-statement-sitemap2.xml',
 'https://www.whitehouse.gov/briefing-statement-sitemap3.xml',
 'https://www.whitehouse.gov/briefing-statement-sitemap4.xml',
 'https://www.whitehouse.gov/briefing-statement-sitemap5.xml',
 'https://www.whitehouse.gov/briefing-statement-sitemap6.xml',
 'https://www.whitehouse.gov/presidential-action-sitemap1.xml',
 'https://www.whitehouse.gov/presidential-action-sitemap2.xml',
 'https://www.whitehouse.gov/eop-component-sitemap.xml',
 'https://www.whitehouse.gov/microsite-sitemap.xml',
 'https://www.whitehouse.gov/authors-sitemap.xml',
 'https://www.whitehouse.gov/issue-sitemap.xml']

In [8]:
# for i, sitemap in enumerate(sitemaps_unique):
#     df = sitemap_df[sitemap_df['sitemap']==sitemaps_unique[i]]
#     print('Crawling: ', sitemaps_unique[i])
#     adv.crawl(df['loc'], f'crawls/wh_crawl_{i+1}.csv', css_selectors=selectors)

In [9]:
import os
os.listdir('crawls')

['wh_crawl_9.csv',
 'wh_crawl_8.csv',
 'wh_crawl_14.csv',
 'wh_crawl_11.csv',
 'wh_crawl_10.csv',
 'wh_crawl_12.csv',
 'wh_crawl_13.csv',
 'wh_crawl_1.csv',
 'wh_crawl_3.csv',
 'wh_crawl_2.csv',
 'wh_crawl_6.csv',
 'wh_crawl_7.csv',
 'wh_crawl_5.csv',
 'wh_crawl_4.csv']

## `explode`  💣🔥💣🔥💣

Now we have the files ready, but before analyzing them you need to make sure that you master the usage of the pandas `explode` method. (skip if you know it already).  
Let's say you have a DataFrame like this, where multiple values are delimited with `@@` although they each form a single string of characters. So we need to split them and handle each of the elements. 

In [10]:
df = pd.DataFrame({'colors': ['blue@@green@@yellow', 'red@@blue', 'green@@blue@@pink@@orange'],
                   'months': ['Jan', 'Feb', 'Mar']})
df.style.set_caption('How do we count the colors?')

,colors,months
0,blue@@green@@yellow,Jan
1,red@@blue,Feb
2,green@@blue@@pink@@orange,Mar


Let's see what happens when we split that column by `@@`:

In [11]:
df['colors'].str.split('@@')

0          [blue, green, yellow]
1                    [red, blue]
2    [green, blue, pink, orange]
Name: colors, dtype: object

We get the values split correctly, but we end up with lists instead of strings. The lists also have different lengths. So how do we deal with this? How do we count the colors?  
#### `explode`!

In [12]:
df['colors'].str.split('@@').explode() 

0      blue
0     green
0    yellow
1       red
1      blue
2     green
2      blue
2      pink
2    orange
Name: colors, dtype: object

Note that the index values have been duplicated, and this preserves the position of the colors. So even though they each have their own row, we can tell which original row they belong to by looking at their index.  
And now we can count the colors: 

In [13]:
df['colors'].str.split('@@').explode().value_counts()

blue      3
green     2
pink      1
yellow    1
orange    1
red       1
Name: colors, dtype: int64

The `explode` method also works on DataFrames, and in this case you have to specify the column. We first `assign` a new column to `df`, which is basically the colors after splitting: 

In [14]:
df.assign(colors_split=df['colors'].str.split('@@'))

,colors,months,colors_split
0,blue@@green@@yellow,Jan,"[blue, green, yellow]"
1,red@@blue,Feb,"[red, blue]"
2,green@@blue@@pink@@orange,Mar,"[green, blue, pink, orange]"


`explode`:

In [15]:
df.assign(colors_split=df['colors'].str.split('@@')).explode('colors_split')

,colors,months,colors_split
0,blue@@green@@yellow,Jan,blue
0,blue@@green@@yellow,Jan,green
0,blue@@green@@yellow,Jan,yellow
1,red@@blue,Feb,red
1,red@@blue,Feb,blue
2,green@@blue@@pink@@orange,Mar,green
2,green@@blue@@pink@@orange,Mar,blue
2,green@@blue@@pink@@orange,Mar,pink
2,green@@blue@@pink@@orange,Mar,orange


Notice that the original information is preserved. All rows and indexes have been duplicated, and each color still belongs to the same correct row.  

Another way you might want to handle this situation is by converting to a dictionary, mapping months to colors. But watch out, this only works if you have unique keys. If you have duplicates, then you will overwrite the original values, and mess up the data.

In [16]:
month_color =  {month: color.split('@@') for month, color in zip(df['months'], df['colors'])}
month_color

{'Jan': ['blue', 'green', 'yellow'],
 'Feb': ['red', 'blue'],
 'Mar': ['green', 'blue', 'pink', 'orange']}

One final trick to explore, as many of the columns in the dataset will be in this format.  
The dicionary above maps months to colors. It is "month-centric". What if we wanted it to be color-centric? What if we wanted to know in which months blue occurs, and the same for the other colors? In other words, how do we invert the mapping, even though the lengths are different, and there might be duplicates?  

This is one of the interesting things tha can be done with `defaultdict`.  
When instantiating it, you have to specify a function. Then, when you try to assign to a key, if that key exists, the assignment happens, otherwise it will call the function. 

For this case I'll use the `list` function. If the key does not exist then we get the output of calling `list()`:

In [17]:
list()

[]

An empty list will be given, to which we will append the desired item.  
I'm aware that this is not a sufficient explanation if it's your first time with `defaultdict` so please check the documentation and some examples online for more info. Let's see it in action.  
For our `month_color` dictionary, we will loop over the items:

In [18]:
from collections import defaultdict

dd = defaultdict(list)

for month, color_list in month_color.items():
    for color in color_list:
        dd[color].append(month)

In [19]:
dict(dd)

{'blue': ['Jan', 'Feb', 'Mar'],
 'green': ['Jan', 'Mar'],
 'yellow': ['Jan'],
 'red': ['Feb'],
 'pink': ['Mar'],
 'orange': ['Mar']}

Now we have the opposite view of the data, from the point of view of the colors instead of the months.  
There are many cases where this will be encountered in this dataset, for example the `links_url` column will contain many links delimited by `@@` which are mapped to the URL on which they occur:

url | links_url
-----|---------
http://example.com | http://one.com@@http://two.com@@http://three.com
http://example.com/1 | http://one.com/hello@@http://two.com

Let's start!

In [20]:
crawl_df = pd.concat([pd.read_csv('crawls/' + file)
                      for file in os.listdir('crawls')],
                     ignore_index=True)
crawl_df.head(2)

,url,url_redirected_to,title,meta_desc,h1,h2,h3,body_text,size,briefing_title,briefing_date,briefing_body_text,briefing_category,resp_meta_download_timeout,resp_meta_download_slot,resp_meta_download_latency,resp_meta_depth,status,links_url,links_text,links_fragment,links_nofollow,img_src,img_alt,ip_address,crawl_time,resp_headers_content-type,resp_headers_accept-ranges,resp_headers_x-akamai-transformed,resp_headers_vary,resp_headers_date,resp_headers_server-timing,resp_headers_strict-transport-security,resp_headers_x-frame-options,request_headers_accept,request_headers_accept-language,request_headers_user-agent,request_headers_accept-encoding
0,https://www.whitehouse.gov/presidential-actions/,https://www.whitehouse.gov/presidential-actions/,Presidential Actions | The White House@@Open M...,Access all official actions from President Don...,Presidential Actions,NaN,NaN,Skip to content White House Logo Economy Natio...,60573,Presidential Actions,"May 25, 2020@@May 24, 2020@@May 22, 2020@@May ...",NaN,\tImmigration\t@@\tImmigration\t@@\tHealthcare...,180.0,www.whitehouse.gov,0.097878,0,200,https://www.whitehouse.gov/presidential-action...,Skip to content@@\n\t\t\t\t\t\t\n\t\t\t\t\t\t\...,@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@...,False@@False@@False@@False@@False@@False@@Fals...,https://www.whitehouse.gov/wp-content/themes/w...,@@@@White House Logo@@Close Menu@@Open Menu@@C...,104.96.132.190,2020-05-26 23:52:42,text/html; charset=UTF-8,bytes,"9 - 0 pmb=mRUM,1",Accept-Encoding,"Tue, 26 May 2020 23:52:42 GMT","cdn-cache; desc=HIT,edge; dur=1",max-age=31536000 ; includeSubDomains ; preload,SAMEORIGIN,"text/html,application/xhtml+xml,application/xm...",en,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"gzip, deflate"
1,https://www.whitehouse.gov/presidential-action...,https://www.whitehouse.gov/presidential-action...,Presidential Memorandum for the Secretary of S...,Presidential Determination No. 2017-07 M...,Presidential Memorandum for the Secretary of S...,NaN,NaN,Skip to content White House Logo Economy Natio...,56820,Presidential Memorandum for the Secretary of S...,"June 1, 2017",Presidential Determination@@\nNo. 2017-0...,\tForeign Policy\t,180.0,www.whitehouse.gov,0.146314,0,200,https://www.whitehouse.gov/presidential-action...,Skip to content@@\n\t\t\t\t\t\t\n\t\t\t\t\t\t\...,@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@...,False@@False@@False@@False@@False@@False@@Fals...,https://www.whitehouse.gov/wp-content/themes/w...,@@@@menu@@White House Logo@@Close Menu@@Open M...,104.96.132.190,2020-05-26 23:52:42,text/html; charset=UTF-8,bytes,"9 - 0 pmb=mRUM,1",Accept-Encoding,"Tue, 26 May 2020 23:52:42 GMT","cdn-cache; desc=HIT,edge; dur=1",max-age=31536000 ; includeSubDomains ; preload,SAMEORIGIN,"text/html,application/xhtml+xml,application/xm...",en,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"gzip, deflate"


Remember that we requested additional columns to the default ones, and they all start with `briefing_`, so we can easily filter them and take a look. 

In [21]:
crawl_df.filter(regex='briefing')

,briefing_title,briefing_date,briefing_body_text,briefing_category
0,Presidential Actions,"May 25, 2020@@May 24, 2020@@May 22, 2020@@May ...",NaN,\tImmigration\t@@\tImmigration\t@@\tHealthcare...
1,Presidential Memorandum for the Secretary of S...,"June 1, 2017",Presidential Determination@@\nNo. 2017-0...,\tForeign Policy\t
2,Four Nominations Sent to the Senate Today and ...,"August 2, 2017",NOMINATIONS SENT TO THE SENATE:@@Richard Glick...,NaN
3,President Donald J. Trump Announces the Appoin...,"June 1, 2017","Following Senate confirmation on May 25, 2017,...",\tLaw & Justice\t
4,Eight Nominations and One Withdrawal Sent to t...,"August 2, 2017",NOMINATIONS SENT TO THE SENATE:@@Daniel M. Gad...,NaN
...,...,...,...,...
8181,Ending the Economic Harm Caused by Our Immigra...,"February 8, 2018",It is time to create a merit-based immigration...,\tImmigration\t
8182,Photos of the Week,"January 12, 2018",\n\t\t\t@@\n\t\t,NaN
8183,Photos of the Week,"January 19, 2018",\n\t\t\t@@\n\t\t,NaN
8184,Photos of the Week,"January 27, 2018",\n\t\t\t@@\n\t\t,NaN


Now let's take a look at the available columns, and conventions used for naming and storing them:

In [22]:
crawl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8186 entries, 0 to 8185
Data columns (total 38 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   url                                     8186 non-null   object 
 1   url_redirected_to                       8186 non-null   object 
 2   title                                   8186 non-null   object 
 3   meta_desc                               8124 non-null   object 
 4   h1                                      8186 non-null   object 
 5   h2                                      104 non-null    object 
 6   h3                                      120 non-null    object 
 7   body_text                               8186 non-null   object 
 8   size                                    8186 non-null   int64  
 9   briefing_title                          8161 non-null   object 
 10  briefing_date                           7924 non-null   obje

* `url` is the original url sent in the request. In some cases you have redirects, and the URL that you request might be redirected to another one. These URLs will be savd in the `url_redirected_to` column. So `url_redirected_to` is the one we are actually dealing with.  
Let's see if we have any differences between them. We compare the difference using set operations:

In [23]:
set(crawl_df['url']).difference(crawl_df['url_redirected_to'])

set()

And again, in the opposite direction: 

In [24]:
set(crawl_df['url_redirected_to']).difference(crawl_df['url'])

set()

`set()` means the empty set. So the two columns are identical (no redirects), and it doesn't matter which one we use. In case there are redirects you should mainly use the `url_redirected_to`.

### Status codes (200, 301, 302, 403, 404, etc.)
Counting the values of the status codes that we have can immediately show if we have any issues and of what type:

In [25]:
crawl_df['status'].value_counts()

200    8186
Name: status, dtype: int64

It seems all URLs are fine. 

The `size` column shows the size of the pages in bytes. This is not very helpful, and can be misleading. It only measures the text of the response of the page, but does not measure images, or videos, scripts that might be calling certain services, the possibilities are endless. Keep this in mind. 
The `cut` function can help in putting the page sizes into bins. There are different ways of `cut`ting series objects, feel free to explore other ways.

In [26]:
pd.cut(crawl_df['size'], bins=10).value_counts()

(38475.458, 61356.2]    5311
(61356.2, 84010.4]      2299
(84010.4, 106664.6]      372
(106664.6, 129318.8]     131
(129318.8, 151973.0]      35
(151973.0, 174627.2]      20
(174627.2, 197281.4]       8
(197281.4, 219935.6]       5
(242589.8, 265244.0]       3
(219935.6, 242589.8]       2
Name: size, dtype: int64

A more natural way to look at page sizes is to sort them by the size, and then see how the frequency changes as the size increases. 

In [27]:
pd.cut(crawl_df['size'], bins=10).value_counts().sort_index()

(38475.458, 61356.2]    5311
(61356.2, 84010.4]      2299
(84010.4, 106664.6]      372
(106664.6, 129318.8]     131
(129318.8, 151973.0]      35
(151973.0, 174627.2]      20
(174627.2, 197281.4]       8
(197281.4, 219935.6]       5
(219935.6, 242589.8]       2
(242589.8, 265244.0]       3
Name: size, dtype: int64

### \<title> Tags

Let's see first if we have duplicated title tags. Now *we* get to be GSC!

In [28]:
crawl_df['title'].duplicated().mean()

0.20962619105790373

20.9% of the page titles are duplicated. Let's check if they have multiple titles per page. 

> It is usually a good practice always split by `@@` for text columns in case they have multiple values

In [29]:
crawl_df['title'].str.split('@@').str.len().value_counts()[:10]

7     7863
8      129
9       44
10      38
4       34
11      10
16      10
18       7
13       6
15       6
Name: title, dtype: int64

So, we have 7,863 pages with seven title tags each. 

In [30]:
adv.emoji_search('think')

,codepoint,status,emoji,name,group,sub_group
0,1F914,fully-qualified,🤔,thinking face,Smileys & Emotion,face-hand


# 🤔 🤔 🤔 🤔 🤔

In [31]:
crawl_df['title'].str.split('@@', expand=True)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62
0,Presidential Actions | The White House,Open Menu,Open Search,Filter by Issue,Close Filter by Issue,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Presidential Memorandum for the Secretary of S...,Open Menu,Copy URL to your clipboard,Open Search,Share this page on Facebook,Share this page on Twitter,Copy URL to your clipboard,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Four Nominations Sent to the Senate Today and ...,Open Menu,Copy URL to your clipboard,Open Search,Share this page on Facebook,Share this page on Twitter,Copy URL to your clipboard,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,President Donald J. Trump Announces the Appoin...,Open Menu,Copy URL to your clipboard,Open Search,Share this page on Facebook,Share this page on Twitter,Copy URL to your clipboard,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,Eight Nominations and One Withdrawal Sent to t...,Open Menu,Copy URL to your clipboard,Open Search,Share this page on Facebook,Share this page on Twitter,Copy URL to your clipboard,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8181,Ending the Economic Harm Caused by Our Immigra...,Open Menu,Copy URL to your clipboard,Open Search,Share this page on Facebook,Share this page on Twitter,Copy URL to your clipboard,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8182,Photos of the Week | The White House,Open Menu,Copy URL to your clipboard,Open Search,Share this page on Facebook,Share this page on Twitter,Copy URL to your clipboard,Download full size image,Download full size image,Download full size image,Download full size image,Download full size image,Download full size image,Download full size image,Download full size image,Download full size image,Download full size image,Download full size image,Download full size image,Download full size image,Download full size image,Download full size image,Download full size image,Download full size image,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8183,Photos of the Week | The White House,Open Menu,Copy URL t

In [32]:
adv.emoji_search('scream')

,codepoint,status,emoji,name,group,sub_group
0,1F631,fully-qualified,😱,face screaming in fear,Smileys & Emotion,face-concerned


# 😱 😱 😱 😱 😱

Looking more closely at titles, here we split, and then explode, making it easy to count the values of individual title tags. 

In [33]:
(crawl_df
 ['title'].str.split('@@')
 .explode()
 .value_counts()
 .reset_index()
 .rename(columns={'index': 'title_tags', 'title': 'count'})
 [:10])

,title_tags,count
0,Copy URL to your clipboard,16324
1,Share this page on Facebook,8194
2,Share this page on Twitter,8194
3,Open Menu,8161
4,Open Search,8160
5,Download full size image,1099
6,Statement from the Press Secretary | The White...,112
7,Bill Announcement | The White House,96
8,Remarks by President Trump Before Marine One D...,91
9,Press Briefing by Press Secretary Sarah Sander...,75


Another way of looking at titles, is by replacing the `@@` with spaces, and counting individual words in the titles. 

In [34]:
(crawl_df
 ['title'].str.replace('@@', ' ')
 .str.split()
 .explode()
 .value_counts()
 .reset_index()
 .rename(columns={'index': 'title_tags', 'title': 'count'})
 [:15])

,title_tags,count
0,to,18762
1,on,18027
2,this,16394
3,Share,16388
4,page,16388
5,your,16350
6,clipboard,16350
7,Copy,16350
8,Open,16332
9,URL,16324


### \<h1> Tags

Let's do the same with h1 tags:

In [35]:
crawl_df['h1'].str.split('@@').str.len().value_counts()

1    8167
2       7
3       6
6       6
Name: h1, dtype: int64

In almost all pages (8,167) we have one h1 tag.  
Let's see what these tags are, and count them (also checking if there are duplicates): 

In [36]:
(crawl_df
 ['h1'].str.split('@@')
 .explode()
 .value_counts()
 .reset_index()
 .rename(columns={'index': 'h1_tags',
                  'h1': 'count'})[:15])

,h1_tags,count
0,Statement from the Press Secretary,116
1,Bill Announcement,96
2,Remarks by President Trump Before Marine One D...,91
3,Press Briefing by Press Secretary Sarah Sanders,75
4,Text of a Letter from the President to the Spe...,74
5,President Donald J. Trump Announces Intent to ...,74
6,\n\t\t\t\t\t,47
7,President Donald J. Trump Announces Intent to ...,40
8,Statement by the President,38
9,President Donald J. Trump Announces Key Additi...,35


### \<h2> Tags

In [37]:
crawl_df['h2'].str.split('@@').str.len().value_counts()

1.0     71
2.0     14
3.0      4
4.0      3
8.0      2
9.0      2
7.0      2
23.0     1
21.0     1
12.0     1
13.0     1
6.0      1
5.0      1
Name: h2, dtype: int64

### Links (`href`, `text`, `fragment`, and `nofollow`)  
Links come with four attributes by default as their names imply.

In [38]:
crawl_df.filter(regex='links').apply(lambda series: series.str[:25])

,links_url,links_text,links_fragment,links_nofollow
0,https://www.whitehouse.go,Skip to content@@\n\t\t\t\t\t\t\n,@@@@@@@@@@@@@@@@@@@@@@@@@,False@@False@@False@@Fals
1,https://www.whitehouse.go,Skip to content@@\n\t\t\t\t\t\t\n,@@@@@@@@@@@@@@@@@@@@@@@@@,False@@False@@False@@Fals
2,https://www.whitehouse.go,Skip to content@@\n\t\t\t\t\t\t\n,@@@@@@@@@@@@@@@@@@@@@@@@@,False@@False@@False@@Fals
3,https://www.whitehouse.go,Skip to content@@\n\t\t\t\t\t\t\n,@@@@@@@@@@@@@@@@@@@@@@@@@,False@@False@@False@@Fals
4,https://www.whitehouse.go,Skip to content@@\n\t\t\t\t\t\t\n,@@@@@@@@@@@@@@@@@@@@@@@@@,False@@False@@False@@Fals
...,...,...,...,...
8181,https://www.whitehouse.go,Skip to content@@\n\t\t\t\t\t\t\n,@@@@@@@@@@@@@@@@@@@@@@@@@,False@@False@@False@@Fals
8182,https://www.whitehouse.go,Skip to content@@\n\t\t\t\t\t\t\n,@@@@@@@@@@@@@@@@@@@@@@@@@,False@@False@@False@@Fals
8183,https://www.whitehouse.go,Skip to content@@\n\t\t\t\t\t\t\n,@@@@@@@@@@@@@@@@@@@@@@@@@,False@@False@@False@@Fals
8184,https://www.whitehouse.go,Skip to content@@\n\t\t\t\t\t\t\n,@@@@@@@@@@@@@@@@@@@@@@@@@,False@@False@@False@@Fals


Let's take a quick look at the `urlparse` function before we move further:

In [39]:
from urllib.parse import urlparse

url = 'https://www.example.com/category/sub-cat/article-title;some_param?one=1&two=2&three=3#fragment'

parsed = urlparse(url)
parsed

ParseResult(scheme='https', netloc='www.example.com', path='/category/sub-cat/article-title', params='some_param', query='one=1&two=2&three=3', fragment='fragment')

The parsed URL is split into its elements, and those elements can be retreived using dot notation: 

In [40]:
parsed.netloc

'www.example.com'

In [41]:
parsed.query

'one=1&two=2&three=3'

Keep in mind that this function dot not _validate_ URLs. It is only useful when you know that you are dealing with correct URLs which we do in this case because they are extracted from pages. Otherwise you will get meaningless results: 

In [42]:
urlparse('anystring/http://hello')

ParseResult(scheme='', netloc='', path='anystring/http://hello', params='', query='', fragment='')

How many links does each page have on average?

In [43]:
(crawl_df['links_url']
 .str.split('@@')
 .str.len()
 .value_counts()
 .reset_index()
 .rename(columns={'index':'links_on_page',
                  'links_url': 'count'})
 .head(15))

,links_on_page,count
0,52,4212
1,53,3356
2,54,256
3,55,89
4,56,51
5,57,21
6,63,18
7,58,17
8,59,16
9,64,14


So the majority of pages have 52-55 links each. We can now separate them in to internal and external links based on whether or not they contain "https://www.whitehouse.gov": 

In [44]:
internal_links = (crawl_df
                  ['links_url'].str.split('@@')
                  .explode()
                  .apply(lambda s: s if 'https://www.whitehouse.gov' in s else None)
                  .dropna())
internal_links.head()

0    https://www.whitehouse.gov/presidential-action...
0                           https://www.whitehouse.gov
0      https://www.whitehouse.gov/issues/economy-jobs/
0    https://www.whitehouse.gov/issues/national-sec...
0    https://www.whitehouse.gov/issues/budget-spend...
Name: links_url, dtype: object

Since they all point to the same domain, it makes sense to check the `path` attribute and check where those internal links point to.

In [45]:
(pd.Series(urlparse(url).path
           for url in internal_links)
 .value_counts()
 .reset_index()
 .rename(columns={'index': 'path', 0: 'count'})
 .head(50))

,path,count
0,/articles/,8188
1,,8185
2,/news/,8175
3,/remarks/,8175
4,/briefings-statements/,8174
5,/presidential-actions/,8174
6,/the-trump-administration/the-cabinet/,8166
7,/about-the-white-house/the-grounds/,8164
8,/ondcp/,8163
9,/about-the-white-house/tours-events/,8163


The first forty links have pretty much the same number, suddenly going down to 1,721, and further down when it comes to `/issues/` pages. So it is clear that these are the links that are available in all pages (navigation, sidebar, footer, etc.). The manually added special links seem to start at index 47. You can explore these separately if you want.  
Let's do the same with the external links.

In [46]:
external_links = (crawl_df
                  ['links_url'].str.split('@@')
                  .explode()
                  .apply(lambda s: s if 'https://www.whitehouse.gov' not in s else None)
                  .dropna())
external_links

0                              http://www.coronavirus.gov
0                           https://apply.whitehouse.gov/
0                          https://twitter.com/whitehouse
0                    https://www.facebook.com/WhiteHouse/
0                   https://www.instagram.com/whitehouse/
                              ...                        
8185    https://twitter.com/intent/tweet?url=https%3A%...
8185                        https://apply.whitehouse.gov/
8185                       https://twitter.com/whitehouse
8185                 https://www.facebook.com/WhiteHouse/
8185                https://www.instagram.com/whitehouse/
Name: links_url, Length: 58797, dtype: object

58,797 links.  
Here it is easier to check the domains that they point to in order to get an overview. 

In [47]:
external_domains = pd.Series(urlparse(url).netloc for url in external_links).value_counts()
print('external domains linked to: ', external_domains.index.nunique(), '\n')
external_domains[:10]

external domains linked to:  481 



twitter.com                 16503
www.facebook.com            16375
apply.whitehouse.gov         8164
www.instagram.com            8163
www.coronavirus.gov          8162
www.federalregister.gov        82
www.youtube.com                67
www.slideshare.net             62
www.wsj.com                    60
links.govdelivery.com:80       36
dtype: int64

Similar to what we saw with internal links, we have the big ones at the top, suddenly going down to 82. Let's remove those, and check the rest. 

In [48]:
external_links_social = pd.Series(urlparse(url).netloc for url in external_links).value_counts()[:5]
external_links_social

twitter.com             16503
www.facebook.com        16375
apply.whitehouse.gov     8164
www.instagram.com        8163
www.coronavirus.gov      8162
dtype: int64

In [49]:
external_links_nonsocial = (pd.Series(link for link in external_links
                                      if urlparse(link).netloc not in external_links_social))
print('Unique external non-social links:', external_links_nonsocial.nunique(), '\n')
external_links_nonsocial

Unique external non-social links: 1307 



0       http://www.uscourts.gov/judges-judgeships/judi...
1                         http://instagram.com/whitehouse
2                          http://facebook.com/whitehouse
3                                              https://0/
4                                              https://1/
                              ...                        
1425    http://dailycaller.com/2018/01/22/poll-majorit...
1426    https://www.washingtontimes.com/news/2018/jan/...
1427    http://thehill.com/opinion/white-house/371634-...
1428    http://www.washingtonexaminer.com/trumps-immig...
1429    https://www.investors.com/news/economy/ibdtipp...
Length: 1430, dtype: object

1,307 unique links out of a total of 1,430.  
One thing you can do with them is `crawl` them with `follow_links=False`. Then open the file into a DataFrame, and get the URLs of the rows where the status code is greater than 400. For example: 

In [50]:
# adv.crawl(external_links_nonsocial.drop_duplicates(), 'external_links.csv', follow_links=False)
# external_links_df = pd.read_csv('external_links.csv')
# external_links_df[external_links_df['status']>400]['url']

Congratulations! We have just implemented a broken-links checker!

### Anchor Text (`links_text`)

Doing the same as we did with the URLs.

In [51]:
link_text_counts = (crawl_df
                    ['links_text'].str.split('@@')
                    .explode()
                    .str.strip()
                    .value_counts())
link_text_counts[:60].reset_index()

,index,links_text
0,Share this page on Facebook,8188
1,Share this page on Twitter,8188
2,White House Logo,8186
3,Skip to content,8186
4,Twitter,8185
5,Facebook,8177
6,Education,8166
7,Contact,8166
8,Immigration,8166
9,Healthcare,8166


Since these are meaningful phrases, and not URLs, we can count the words that are used in the anchor text using `word_frequency`. 

In [52]:
adv.word_frequency(crawl_df
                   ['links_text'].str.split('@@')
                   .explode()
                   .str.strip()).iloc[:20, :2]

,word,abs_freq
0,house,40877
1,white,40867
2,policy,34393
3,national,24602
4,share,24557
5,office,24522
6,council,24499
7,&,17707
8,president,16560
9,trump,16518


"Words" don't necessarily have to be single words. They can be phrases consisting of two or more words.  
The `word_frequency` function has the option of specifying this through the `phrase_len` parameter. Now we can take a look at phrases consisting of two words (bigrams is the technical term).

In [53]:
adv.word_frequency(crawl_df                   
                   ['links_text'].str.split('@@')
                   .explode()
                   .str.strip(),
                   phrase_len=2).iloc[:20, :2]

,word,abs_freq
0,white house,40866
1,office of,24486
2,page on,16378
3,share this,16376
4,this page,16376
5,national security,16340
6,the white,16338
7,to your,16323
8,your clipboard,16323
9,copy url,16299


### Images

In [54]:
(crawl_df['img_src']
 .str.split('@@').str.len()
 .value_counts()
 .reset_index()
 .rename(columns={'index':'images_on_page',
                  'img_src': 'count'})
 .head(15))

,images_on_page,count
0,9,7317
1,10,406
2,11,204
3,8,64
4,12,56
5,13,22
6,15,13
7,14,11
8,2,9
9,25,7


Sorting the index to see the frequency by the number of images on the pages (as opposed to sorted by the count).

In [55]:
(crawl_df['img_src']
 .str.split('@@').str.len()
 .value_counts()
 .reset_index()
 .rename(columns={'index':'images_on_page',
                  'img_src': 'count'})
 .sort_values('images_on_page')
 .head(15)
)

,images_on_page,count
8,2,9
19,3,3
29,5,2
10,6,6
41,7,1
3,8,64
0,9,7317
1,10,406
2,11,204
4,12,56


So we can see that most pages have between eight and twelve images.

### Response Headers
These columns all start with `resp_headers_` and vary from site to site. Sometimes you get five or six columns, and sometimes many more.

In [56]:
crawl_df.filter(regex='resp_headers').sample(5)

,resp_headers_content-type,resp_headers_accept-ranges,resp_headers_x-akamai-transformed,resp_headers_vary,resp_headers_date,resp_headers_server-timing,resp_headers_strict-transport-security,resp_headers_x-frame-options
3972,text/html; charset=UTF-8,bytes,"9 54099 0 pmb=mRUM,1",Accept-Encoding,"Tue, 26 May 2020 23:47:37 GMT","cdn-cache; desc=REVALIDATE,edge; dur=146,origi...",max-age=31536000 ; includeSubDomains ; preload,SAMEORIGIN
8015,text/html; charset=UTF-8,bytes,"9 - 0 pmb=mRUM,1",Accept-Encoding,"Tue, 26 May 2020 23:48:37 GMT","cdn-cache; desc=REVALIDATE,edge; dur=140,origi...",max-age=31536000 ; includeSubDomains ; preload,SAMEORIGIN
7781,text/html; charset=UTF-8,bytes,"9 - 0 pmb=mRUM,1",Accept-Encoding,"Tue, 26 May 2020 23:48:22 GMT","cdn-cache; desc=REVALIDATE,edge; dur=131,origi...",max-age=31536000 ; includeSubDomains ; preload,SAMEORIGIN
3312,text/html; charset=UTF-8,bytes,"9 - 0 pmb=mRUM,1",Accept-Encoding,"Tue, 26 May 2020 23:46:55 GMT","cdn-cache; desc=REVALIDATE,edge; dur=133,origi...",max-age=31536000 ; includeSubDomains ; preload,SAMEORIGIN
2564,text/html; charset=UTF-8,bytes,"9 57133 0 pmb=mRUM,1",Accept-Encoding,"Tue, 26 May 2020 23:45:52 GMT","cdn-cache; desc=REVALIDATE,edge; dur=135,origi...",max-age=31536000 ; includeSubDomains ; preload,SAMEORIGIN


### Request Headers
This depends on how we are running the crawl and what parameters were specified. All request headers can be customized by using the `custom_settings` parameter in the `crawl` function. 

`adv.crawl('https://example.com', 'output_file.csv', cutom_settings={'DEFAULT_REQUEST_HEADERS': {'User-Agent': 'your_user_agent'}} `

In [57]:
crawl_df.filter(regex='request_headers').sample(5)

,request_headers_accept,request_headers_accept-language,request_headers_user-agent,request_headers_accept-encoding
5116,"text/html,application/xhtml+xml,application/xm...",en,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"gzip, deflate"
7326,"text/html,application/xhtml+xml,application/xm...",en,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"gzip, deflate"
7418,"text/html,application/xhtml+xml,application/xm...",en,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"gzip, deflate"
3952,"text/html,application/xhtml+xml,application/xm...",en,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"gzip, deflate"
2240,"text/html,application/xhtml+xml,application/xm...",en,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"gzip, deflate"


These were some options that you might want to start with. What you do next depends on what you are tracking, the cutom data you extracted, and you strategy. 

Your turn. 